In [41]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re
import random

In [42]:
main_url = "https://stats.espncricinfo.com/ci/engine/stats/index.html?class=11;filter=advanced;orderby=runs;size=150;team=8;template=results;type=batting"

In [43]:
response = requests.get(
    url = main_url,
)

main_soup = soup(response.content, 'html.parser')

In [44]:
cricketers_list_raw = main_soup.find_all('tr', "data1")

In [53]:
cricketers_list = []
for i in range(0,100):
    for x in cricketers_list_raw[i].find_all("a", "data-link"):
        cricketers_list.append({"name": x.getText(), "url": x.get("href")})
    

In [60]:
# cricketers_list

In [55]:
base_url_player = "https://stats.espncricinfo.com/"

In [83]:
def extract_player_data(soup):
    player_card = soup.find_all('div', "player_overview-grid")
    player_details_raw = player_card[0].find_all('div')
    player_details = []
    for detail in player_details_raw:
        player_details.append({"field": detail.p.getText(), "value": detail.h5.getText()})
    print(player_details)
    

In [84]:
for p in cricketers_list:
    player_url = base_url_player + p["url"]
    response = requests.get(
    url = player_url,
    )
    player_soup = soup(response.content, 'html.parser')
    extract_player_data(player_soup)
    break

[{'field': 'Full Name', 'value': 'Kumar Chokshanada Sangakkara'}, {'field': 'Born', 'value': 'October 27, 1977, Matale'}, {'field': 'Age', 'value': '43y 361d'}, {'field': 'Batting Style', 'value': 'Left hand bat'}, {'field': 'Bowling Style', 'value': 'Right arm offbreak'}, {'field': 'Fielding Position', 'value': 'Wicketkeeper'}, {'field': 'Playing Role', 'value': 'Wicketkeeper batter'}]
1
